In [ ]:
%%capture
%config Completer.use_jedi = False
%config InlineBackend.figure_formats = ['svg']

# Install on Google Colab
import subprocess
import sys

from IPython import get_ipython

install_packages = "google.colab" in str(get_ipython())
if install_packages:
    for package in ["ampform", "graphviz"]:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", package]
        )

# Standard lineshapes

AmpForm provides a few common lineshapes through the {mod}`.dynamics` module. This pages shows some of the properties of these lineshapes. It's also possible to insert {doc}`custom lineshapes </usage/dynamics/custom>` into the amplitude model.

In [ ]:
import sympy as sp

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from qrules.combinatorics import arange


def plot_real_imag(
    expression: sp.Expr,
    variable: sp.Symbol,
    x_min: float,
    x_max: float,
    resolution: int = 100,
) -> Figure:
    delta = (x_max - x_min) / resolution
    x = list(arange(x_min, x_max, delta))
    y_real = list(map(lambda x: sp.Abs(expression).subs(variable, x), x))
    y_imag = list(map(lambda x: sp.arg(expression).subs(variable, x), x))
    fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(8, 8))
    for a in ax:
        a.xaxis.set_ticks([])
        a.yaxis.set_ticks([])
    ax_real, ax_imag = ax
    ax_imag.set(xlabel=f"${variable.name}$")
    ax_imag.set(ylabel=f"imag $f({variable.name})$")
    ax_real.set(ylabel=f"real $f({variable.name})$")
    ax_real.yaxis.set_ticks([])
    ax_imag.yaxis.set_ticks([0, float(sp.pi)])
    ax_imag.yaxis.set_ticklabels([0, R"$\pi$"])
    ax_imag.set_ylim([0, float(sp.pi)])
    ax_real.plot(x, y_real)
    ax_imag.plot(x, y_imag)
    return fig

## Form factor

AmpForm uses {class}`.BlattWeisskopf` functions $B_L$ as _barrier factors_ (also called _form factors_):

In [ ]:
from ampform.dynamics import BlattWeisskopf

L = sp.Symbol("L", integer=True)
z = sp.Symbol("z", real=True)
ff2 = BlattWeisskopf(L, z) ** 2
display(ff2, ff2.doit())

## Relativistic Breit-Wigner

### _Without_ form factor

See {func}`.relativistic_breit_wigner`:

In [ ]:
from ampform.dynamics import relativistic_breit_wigner

m, m0, w0 = sp.symbols("m m0 Gamma", real=True)
relativistic_breit_wigner(m, m0, w0)

In [ ]:
plot_real_imag(relativistic_breit_wigner(m, 1.0, 0.3), m, x_min=0, x_max=2);

### _With_ form factor

See {func}`.relativistic_breit_wigner_with_ff`. For $L=0$, the shape is:

In [ ]:
from ampform.dynamics import relativistic_breit_wigner_with_ff

m, m0, w0, ma, mb, d = sp.symbols("m m0 Gamma m_a m_b d", real=True)
relativistic_breit_wigner_with_ff(
    mass=m,
    mass0=m0,
    gamma0=w0,
    m_a=ma,
    m_b=mb,
    angular_momentum=0,
    meson_radius=d,
).doit()

In [ ]:
ma = 0.2
mb = 0.3
complex_bw_ff = relativistic_breit_wigner_with_ff(
    mass=m,
    mass0=1.0,
    gamma0=0.3,
    m_a=ma,
    m_b=mb,
    angular_momentum=0,
    meson_radius=1,
)
plot_real_imag(complex_bw_ff.doit(), m, x_min=ma + mb, x_max=2);